In [1]:
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.distributed import *
from fastai.vision.models.xresnet import *
from fastai.vision.models.unet import DynamicUnet
from bpho import *

/home/fredmonroe/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
datasetname = 'combo_001'
data_path = Path('.')
datasets = data_path/'datasets'
datasources = data_path/'data'
dataset = datasets/datasetname

hr_tifs = dataset/'hr'
lr_tifs = dataset/'lr'
lr_up_tifs = dataset/'lr_up'

mname = 'combo'
model_dir = 'models'

loss = F.mse_loss
metrics = sr_metrics

In [3]:
def get_src(x_data, y_data_):
    def map_to_hr(x):
        hr_name = x.relative_to(x_data)
        return y_data_/hr_name
    src = (ImageImageList
            .from_folder(x_data, convert_mode='L')
            .split_by_rand_pct()
            .label_from_func(map_to_hr, convert_mode='L'))
    return src


def get_data(bs, size, x_data, y_data, max_zoom=1.1):
    src = get_src(x_data, y_data)
    tfms = get_transforms(flip_vert=True, max_zoom=max_zoom)
    data = (src
            .transform(tfms, size=size)
            .transform_y(tfms, size=size)
            .databunch(bs=bs))
    data.c = 3
    return data

def do_fit(learn, save_name, lrs=slice(1e-3), pct_start=0.9, cycle_len=10):
    learn.to_fp16().fit_one_cycle(cycle_len, lrs, pct_start=pct_start)
    learn.save(save_name)
    print(f'saved: {save_name}')
    num_rows = min(learn.data.batch_size, 3)
    learn.show_results(rows=num_rows, imgsize=5)

In [ ]:
if True:
    learn.lr_find()
    learn.recorder.plot()

In [ ]:
step = 0
lr = 1e-3
cycles = 5

bs = 300
size = 64
max_zoom = 8
arch = xresnet34

data = get_data(bs, size, hr_tifs, lr_up_tifs, max_zoom=max_zoom)
learn = xres_unet_learner(data, arch, loss_func=loss, metrics=metrics, model_dir=model_dir)
gc.collect()

do_fit(learn, f'{mname}.{step:02d}', lrs=lr, cycle_len=cycles)

In [ ]:
step = 1
lr = 1e-3
cycles = 3

bs = 20
size = 256
max_zoom = 8
arch = xresnet34

data = get_data(bs, size, hr_tifs, lr_up_tifs, max_zoom=max_zoom)
learn = xres_unet_learner(data, arch, loss_func=loss, metrics=metrics, model_dir=model_dir)
gc.collect()

if Path(f'{mname}.{(step-1):02d}').exists(): 
    print('loading', f'{mname}.{(step-1):02d}')
    learn.load(f'{mname}.{(step-1):02d}')

do_fit(learn, f'{mname}.{step:02d}', lrs=lr, cycle_len=cycles)

In [4]:
movie_files = []
movie_files = list(Path('/scratch/bpho/datasets/movies_001/test').glob('*05*.czi'))
#movie_files += list(Path('/scratch/bpho/datasources/low_res_test/').glob('low res confocal*.czi'))
#movie_files += list(Path('/scratch/bpho/datasources/neuron_movies2/').glob('low*.*'))
#movie_files += list(Path('/DATA/WAMRI/salk/uri/bpho/datasources/neuron_movies/').glob('low res 300 time points 2*.czi'))
#movie_files += list(Path('/DATA/WAMRI/salk/uri/bpho/datasources/neuron_movies/').glob('*time points 2*.tif'))
#movie_files = list(Path('/DATA/donow/').glob('*.czi'))

In [5]:
step = 1
lr = 1e-3
cycles = 3

bs = 1
size = 256*6
max_zoom = 1
arch = xresnet34

data = get_data(bs, size, hr_tifs, lr_up_tifs, max_zoom=max_zoom)
learn = xres_unet_learner(data, arch, loss_func=loss, metrics=metrics, model_dir=model_dir)
gc.collect()

print(model_dir)
print(mname)
learn = learn.load('/home/fredmonroe/repos/salk/uri/paper/datasets/combo_001/lr_up/models/combo.1')
learn = learn.to_fp16()

models
combo


In [6]:
generate_movies(movie_files, learn, size, wsize=1)

czi MitoTracker Red FM 230x230 10min 02-Airyscan Processing-05
czi: x:228 y:230 t:300 z:1
7.8196077
